# Experiments

In [38]:
dataset_name = 'cora'

## Global arguments

In [39]:
import torch
import argparse
import numpy as np
import torch.optim as optim
import torch.nn as nn
import sklearn
from kcenter import make_all_dists, greedy_kcenter, gonzalez_kcenter, CenterObjective, make_dists_igraph, rounding
from models import GCNLink, GCNClusterNet, GCNDeep, GCNDeepSigmoid, GCN
from utils import make_normalized_adj, negative_sample, edge_dropout, load_nofeatures, accuracy, calculate_accuracy
from modularity import baseline_spectral, partition, greedy_modularity_communities, make_modularity_matrix
from loss_functions import loss_kcenter, loss_modularity

import matplotlib.pyplot as plt

In [40]:
normalize_features = True
# For some reason the original code provides its own load_data function but uses the
# function from pygcn, which makes some feature normaliztion with sparse matrix.
if(normalize_features):
    from pygcn import load_data
else:
    from utils import load_data

In [41]:
# class Object(object):
#     pass

# args = Object()

class AttrDict(object):
    def __init__(self, d):
        self.__dict__ = d

no_cuda = True

args = AttrDict(dict(
    dataset = dataset_name,
    train_pct = 0.40,
    objective = 'modularity',
    hidden = 50,
    embed_dim = 50,
    dropout = 0.2,
    edge_dropout = 0.2,
    K = 7,
    train_iters = 1001,
    lr = 0.001,
    num_cluster_iter = 1,
    weight_decay = 5e-4,
    pure_opt = True,
    clustertemp = 70,
    kcentertemp = 100,
    kcentermintemp = 0,
    seed = 24,
    negsamplerate = 1,
    cuda = not no_cuda and torch.cuda.is_available(),
    use_igraph = True,
))

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

run_decision = True
run_ts = True
run_gcne2e = True
run_train_only = True
calculate_opt = True
    
pure_opt = args.pure_opt
test_cluster_auc = False

reload_data = True
make_objectives = False
if reload_data:
    make_objectives = True

has_features = True

train_pct = args.train_pct

## Preparations

### Load data

In [42]:
if reload_data:
    if has_features:
        adj_test, features_test, labels, idx_train, idx_val, idx_test = load_data('data/{}/'.format(args.dataset), '{}_test_{:.2f}'.format(args.dataset, train_pct))
        adj_valid, features_valid, labels, idx_train, idx_val, idx_test = load_data('data/{}/'.format(args.dataset), '{}_valid_{:.2f}'.format(args.dataset, train_pct))
        adj_train, features_train, labels, idx_train, idx_val, idx_test = load_data('data/{}/'.format(args.dataset), '{}_train_{:.2f}'.format(args.dataset, train_pct))
    else:
        adj_all, features, labels = load_nofeatures(args.dataset, '')
        features_train = features
        features_test = features
        n = adj_all.shape[0]
        adj_train, features, labels = load_nofeatures(args.dataset, '_train_{:.2f}'.format(train_pct), n)
        adj_test, features, labels = load_nofeatures(args.dataset, '_test_{:.2f}'.format(train_pct), n)
        adj_valid, features, labels = load_nofeatures(args.dataset, '_valid_{:.2f}'.format(train_pct), n)


adj_test = adj_test.coalesce()
adj_valid = adj_valid.coalesce()
adj_train = adj_train.coalesce()
n = adj_train.shape[0]
K = args.K
bin_adj_test = (adj_test.to_dense() > 0).float()
bin_adj_train = (adj_train.to_dense() > 0).float()
m_train = bin_adj_train.sum()
bin_adj_valid = (adj_valid.to_dense() > 0).float()
bin_adj_all = (bin_adj_train + bin_adj_test + bin_adj_valid > 0).float()
adj_all = make_normalized_adj(bin_adj_all.nonzero(), n)
nfeat = features_test.shape[1]

adj_all, features_test, labels, idx_train, idx_val, idx_test = load_data('data/{}/'.format(args.dataset), '{}'.format(args.dataset))
adj_all = adj_all.coalesce()
adj_test = adj_all
bin_adj_all = (adj_all.to_dense() > 0).float()
n = adj_all.shape[0]
K= args.K
nfeat = features_test.shape[1]

Loading cora_test_0.40 dataset...
Loading cora_valid_0.40 dataset...
Loading cora_train_0.40 dataset...
Loading cora dataset...


In [43]:
def plot_if_possible(r=None, print_normalized=False):
    if features_train.shape[1] == 2:
        if normalize_features and not print_normalized:
            x = []
            y = []
            with open("data/%s/%s.content" % (args.dataset, args.dataset)) as ss:
                for line in ss:
                    _, xx, yy, _ = line.split(' ')
                    x.append(float(xx))
                    y.append(float(yy))
        else:
            x = features_train[:,0]
            y = features_train[:,1]
        
        if print_normalized:
            text = " normalized"
        else:
            text = ""
        if r is None:
            plt.scatter(x, y, c=labels, s=5)
            plt.title("The%s dataset with true labels" % text)
        else:
            predictions = r.argmax(dim=1)
            plt.scatter(x, y, c=predictions, s=5)
            plt.title("The%s dataset with predicted labels" % text)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.show()

In [44]:
plot_if_possible()

In [45]:
if normalize_features:
    plot_if_possible(print_normalized=True)

### Initialize models

In [46]:
model_ts = GCNLink(nfeat=nfeat,
            nhid=args.hidden,
            nout=args.embed_dim,
            dropout=args.dropout)

model_cluster = GCNClusterNet(nfeat=nfeat,
            nhid=args.hidden,
            nout=args.embed_dim,
            dropout=args.dropout,
            K = args.K,
            cluster_temp = args.clustertemp)

#keep a couple of initializations here so that the random seeding lines up
#with results reported in the paper -- removing these is essentially equivalent to 
#changing the seed
_ = GCN(nfeat, args.hidden, args.embed_dim, args.dropout)
_ = nn.Parameter(torch.rand(K, args.embed_dim))

#uses GCNs to predict the cluster membership of each node
model_gcn = GCNDeep(nfeat=nfeat,
            nhid=args.hidden,
            nout=args.K,
            dropout=args.dropout,
            nlayers=2)

#uses GCNs to predict the probability that each node appears in the solution
model_gcn_x = GCNDeepSigmoid(nfeat=nfeat,
            nhid=args.hidden,
            nout=1,
            dropout=args.dropout,
            nlayers=2)

if args.objective == 'kcenter':
    model_gcn = model_gcn_x



optimizer = optim.Adam(model_cluster.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

if args.cuda:
    model_cluster.cuda()
    model_ts.cuda()
    features = features.cuda()
    adj_train = adj_train.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

losses = []
losses_test = []
num_cluster_iter = args.num_cluster_iter

### Auxilary data for objectives

In [47]:
if make_objectives:
    if args.objective == 'kcenter':
        try:
            dist_all = torch.load('{}_test_dist.pt'.format(args.dataset))
            dist_train = torch.load('{}_{}_train_dist.pt'.format(args.dataset, train_pct))
            diameter = dist_all.max()
        except:
            dist_all = make_all_dists(bin_adj_all, 100)
            diameter = dist_all[dist_all < 100].max()
            dist_all[dist_all == 100] = diameter
            torch.save(dist_all, '{}_test_dist.pt'.format(args.dataset))
            dist_train = make_all_dists(bin_adj_train, 100)
            dist_train[dist_train == 100] = diameter
            torch.save(dist_train, '{}_{}_train_dist.pt'.format(args.dataset, train_pct))
        obj_train = CenterObjective(dist_train, diameter, args.kcentermintemp)
        obj_train_hardmax = CenterObjective(dist_train, diameter, args.kcentermintemp, hardmax=True)
        obj_test = CenterObjective(dist_all, diameter, args.kcentertemp, hardmax=True)
        obj_test_softmax = CenterObjective(dist_all, diameter, args.kcentermintemp)

    if args.objective == 'modularity':
        mod_train = make_modularity_matrix(bin_adj_train)
        mod_test = make_modularity_matrix(bin_adj_test)
        mod_valid = make_modularity_matrix(bin_adj_valid)
        mod_all = make_modularity_matrix(bin_adj_all)

### Define loss functions

In [48]:
if args.objective == 'modularity':
    loss_fn = loss_modularity
    test_object = mod_all
    train_object = mod_train
    test_only_object = mod_test
    valid_object = mod_valid
elif args.objective == 'kcenter':
    loss_fn = loss_kcenter
    test_object= obj_test
    train_object = obj_train
    test_only_object = None
    valid_object = None
else:
    raise Exception('unknown objective')

## Training

### Decision-Focused

In [49]:
best_train_val = 100
if run_decision:
    for t in range(args.train_iters):
        #pure optimization setting: get loss with respect to the full graph
        if pure_opt:
            mu, r, embeds, dist = model_cluster(features_test, adj_all, num_cluster_iter)
            loss = loss_fn(mu, r, embeds, dist, bin_adj_all, test_object, args)
        #link prediction setting: get loss with respect to training edges only
        else:
            mu, r, embeds, dist = model_cluster(features_train, adj_train, num_cluster_iter)
            loss = loss_fn(mu, r, embeds, dist, bin_adj_train, train_object, args)
        if args.objective != 'kcenter':
            loss = -loss
        optimizer.zero_grad()
        loss.backward()
        #increase number of clustering iterations after 500 updates to fine-tune
        #solution
        if t == 500:
            num_cluster_iter = 5
        #every 100 iterations, look and see if we've improved on the best training loss
        #seen so far. Keep the solution with best training value.
        if t % 100 == 0:
            #round solution to discrete partitioning
            if args.objective == 'modularity':
                r = torch.softmax(100*r, dim=1)
            #evalaute test loss -- note that the best solution is
            #chosen with respect training loss. Here, we store the test loss
            #of the currently best training solution
            loss_test = loss_fn(mu, r, embeds, dist, bin_adj_all, test_object, args)
            #for k-center problem, keep track of the fractional x with best
            #training loss, to do rounding after
            if loss.item() < best_train_val:
                best_train_val = loss.item()
                curr_test_loss = loss_test.item()
                #convert distances into a feasible (fractional x)
                x_best = torch.softmax(dist*args.kcentertemp, 0).sum(dim=1)
                x_best = 2*(torch.sigmoid(4*x_best) - 0.5)
                if x_best.sum() > K:
                    x_best = K*x_best/x_best.sum()
        losses.append(loss.item())
        optimizer.step()

    #for k-center: round 50 times and take the solution with best training
    #value
    if args.objective == 'kcenter':
        testvals = []; trainvals = []
        for _ in range(50):
            y = rounding(x_best)
            testvals.append(obj_test(y).item())
            trainvals.append(obj_train(y).item())
        print('ClusterNet value', testvals[np.argmin(trainvals)])
    if args.objective == 'modularity':
            print('ClusterNet value', curr_test_loss)
            tmp_training_state = model_cluster.training
            model_cluster.training = False
            mu, r, embeds, dist = model_cluster(features_train, adj_train, num_cluster_iter)
            model_cluster.training = tmp_training_state
            print("ClusterNet accuracy: ", accuracy(r, labels).item())
            plot_if_possible(r)

ClusterNet value 0.5756863355636597
ClusterNet accuracy:  0.04689807976366322


### Two-stages

In [50]:
def train_twostage(model_ts):
    optimizer_ts = optim.Adam(model_ts.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)
    edges = adj_train.indices().t()
    edges_test = adj_test.indices().t()
    edges_test_eval, labels_test_eval = negative_sample(edges_test, 1, bin_adj_train)
#    print(edges_test_eval)
    for t in range(300):
        adj_input = make_normalized_adj(edge_dropout(edges, args.edge_dropout), n)
        edges_eval, labels = negative_sample(edges, args.negsamplerate, bin_adj_train)
        preds = model_ts(features_train, adj_input, edges_eval)
        loss = torch.nn.BCEWithLogitsLoss()(preds, labels)
        optimizer_ts.zero_grad()
        loss.backward()
        if t % 10 == 0:
            preds_test_eval = model_ts(features_train, adj_input, edges_test_eval)
            test_ce = torch.nn.BCEWithLogitsLoss()(preds_test_eval, labels_test_eval)
            test_auc = sklearn.metrics.roc_auc_score(labels_test_eval.long().detach().numpy(), nn.Sigmoid()(preds_test_eval).detach().numpy())
            print(t, loss.item(), test_ce.item(), test_auc)
        optimizer_ts.step()

if test_cluster_auc:
    model_linkpred = GCNLink(nfeat=nfeat,
            nhid=args.hidden,
            nout=args.embed_dim,
            dropout=args.dropout)
    model_linkpred.GCN = model_cluster.GCN
    model_linkpred.GCN.requires_grad = False
    train_twostage(model_linkpred)


calculate_ts_performance = False
if run_ts:
    print('two stage')
    train_twostage(model_ts)
    #predict probability that all unobserved edges exist
    indices = torch.tensor(np.arange(n))
    to_pred = torch.zeros(n**2, 2)
    to_pred[:, 1] = indices.repeat(n)
    for i in range(n):
        to_pred[i*n:(i+1)*n, 0] = i
    to_pred = to_pred.long()
    preds = model_ts(features_train, adj_train, to_pred)
    preds = nn.Sigmoid()(preds).view(n, n)

    preds = bin_adj_train + (1 - bin_adj_train)*preds

    if args.objective == 'modularity':
        r = greedy_modularity_communities(preds, K)
        print('agglomerative', loss_fn(None, r, None, None, bin_adj_all, test_object, args))
        print("agglomerative accuracy: ", accuracy(r, labels).item())
        plot_if_possible(r)
        
        r = partition(preds, K)
        print('recursive', loss_fn(None, r, None, None, bin_adj_all, test_object, args))
        print("recursive accuracy: ", accuracy(r, labels).item())
        plot_if_possible(r)

        degrees = preds.sum(dim=1)
        preds = torch.diag(1./degrees)@preds
        mod_pred = make_modularity_matrix(preds)
        r = baseline_spectral(mod_pred, K)
        print('spectral', loss_fn(None, r, None, None, bin_adj_all, test_object, args))
        print("spectral accuracy: ", accuracy(r, labels).item())
        plot_if_possible(r)
    elif args.objective == 'kcenter':
        try:
            dist_ts = torch.load('{}_twostage_dist.pt'.format(args.dataset))
            print('loaded ts dists from {}'.format('{}_twostage_dist.pt'.format(args.dataset)))
        except:
            print('making dists')
            if args.use_igraph:
                print('using igraph')
                dist_ts =  make_dists_igraph(preds)
            else:
                print('using networkx')
                dist_ts = make_all_dists(preds, 100)
                diameter = dist_ts[dist_ts < 100].max()
                dist_ts[dist_ts == 100] = diameter
            print('made dists')
            torch.save(dist_ts, '{}_twostage_dist.pt'.format(args.dataset))
        dist_ts = dist_ts.float()
        diameter = dist_ts.max()
        x = gonzalez_kcenter(dist_ts, K)
        print('gonzalez ts', obj_train_hardmax(x), obj_test(x))
        print(dist_ts.type(), diameter.type())
        x = greedy_kcenter(dist_ts, diameter, K)
        print('greedy ts', obj_train_hardmax(x), obj_test(x))

two stage
0 0.6976549029350281 0.6979550719261169 0.550937959604964
10 0.6936492323875427 0.6939929723739624 0.5672309299976281
20 0.692654550075531 0.6930332183837891 0.57672392672294
30 0.6924092769622803 0.6927725076675415 0.6020536509390446
40 0.6922897100448608 0.6926782727241516 0.6283998148939018
50 0.6922233700752258 0.6926360726356506 0.6386608879063167
60 0.6921097636222839 0.6926025152206421 0.6548568245846436
70 0.6919981837272644 0.692530632019043 0.677261648615624
80 0.6918414831161499 0.6924417018890381 0.6804755014070756
90 0.6916250586509705 0.6923367381095886 0.687436373757532
100 0.6913665533065796 0.6922526359558105 0.6693443875583673
110 0.6910643577575684 0.6920222640037537 0.6686139163472129
120 0.6904924511909485 0.6917594075202942 0.6634707941337172
130 0.6898487210273743 0.6913962364196777 0.6592036922818925
140 0.6889950633049011 0.6908406615257263 0.6703813520748181
150 0.6875816583633423 0.6900725960731506 0.6670087718727855
160 0.6858158707618713 0.6890211

### E2E GCN

In [51]:
if run_gcne2e:
    print('just GCN')
    optimizer_gcn = optim.Adam(model_gcn.parameters(), lr = args.lr,
                               weight_decay = args.weight_decay)
    if args.objective == 'modularity':
        best_train_val = 0
    if args.objective == 'kcenter':
        best_train_val = 100

    for t in range(1000):
        best_train_loss = 100
        if pure_opt:
            if args.objective == 'modularity' or args.objective == 'maxcut':
                r = model_gcn(features_test, adj_all)
                r = torch.softmax(args.clustertemp*r, dim = 1)
                loss = -loss_fn(None, r, None, None, bin_adj_train, train_object, args)
            elif args.objective == 'kcenter' or args.objecive == 'influmax':
                x = model_gcn(features_test, adj_all)
                if x.sum() > K:
                    x = K*x/x.sum()
                loss = -test_object(x)
        else:
            if args.objective == 'modularity' or args.objective == 'maxcut':
                r = model_gcn(features_train, adj_train)
                r = torch.softmax(r, dim = 1)
                loss = -loss_fn(None, r, None, None, bin_adj_train, train_object, args)
            elif args.objective == 'kcenter' or args.objecive == 'influmax':
                x = model_gcn(features_train, adj_train)
                if x.sum() > K:
                    x = K*x/x.sum()
                loss = -train_object(x)
        if args.objective == 'kcenter':
            loss = -loss
        optimizer.zero_grad()
        loss.backward()
        if t % 100 == 0:
            if args.objective == 'modularity' or args.objective == 'maxcut':
                r = torch.softmax(100*r, dim=1)
                loss_test = loss_fn(None, r, None, None, bin_adj_all, test_object, args)
                loss_test_only = loss_fn(None, r, None, None, bin_adj_test, test_only_object, args)
            elif args.objective == 'kcenter' or args.objecive == 'influmax':
                loss_test = -test_object(x)
                loss_test_only = torch.tensor(0).float()
            losses_test.append(loss_test.item())
            print(t, loss.item(), loss_test.item(), loss_test_only.item())
            if loss.item() < best_train_val:
                curr_test_loss = loss_test.item()
                best_train_val = loss.item()
                if args.objective == 'kcenter' or args.objective == 'influmax':
                    x_best = x
        losses.append(loss.item())
        optimizer.step()
    if args.objective == 'kcenter':
        from influmax import rounding
        testvals = []; trainvals = []; trainvalshardmax = []
        for _ in range(50):
            y = rounding(x_best)
            testvals.append(obj_test(y).item())
            trainvals.append(obj_train(y).item())
            trainvalshardmax.append(obj_train_hardmax(y).item())
        print('train min', testvals[np.argmin(trainvals)])
        print('hardmax train min', testvals[np.argmin(trainvalshardmax)])
        print('absolute min', min(testvals))
    if args.objective == 'modularity':
        print('train min', curr_test_loss)
        print("e2e gcn accuracy: ", accuracy(r, labels).item())
        plot_if_possible(r)

just GCN
0 -0.0003772028721868992 -1.2361405197225395e-06 1.1375951913805693e-07
100 -0.0018916751723736525 0.003027205355465412 0.003719148226082325
200 -0.00015944383630994707 0.0003794107760768384 0.0007440358749590814
300 -0.00020667437638621777 0.00037874022382311523 0.0007424590876325965
400 -0.00026252202223986387 0.001134658814407885 0.0022295506205409765
500 -0.00043619528878480196 0.00037388905184343457 0.0007375568384304643
600 -0.0010287465993314981 0.0003778848913498223 -1.218813508785388e-06
700 -0.0010210201144218445 0.0030236588791012764 0.0037156138569116592
800 -0.00028532487340271473 0.0006094782147556543 0.0011970620835199952
900 -0.0006831673672422767 6.367894457071088e-06 1.2431093637133017e-05
train min 0.003027205355465412
e2e gcn accuracy:  0.15472673559822747


### Train-only baseline

In [52]:
if run_train_only:
    if args.objective == 'modularity':
        preds = bin_adj_train
        r = greedy_modularity_communities(preds, K)
        print('agglomerative', loss_fn(None, r, None, None, bin_adj_all, test_object, args))
        print("agglomerative accuracy: ", accuracy(r, labels).item())
        plot_if_possible(r)

        r = partition(preds, K)
        print('recursive', loss_fn(None, r, None, None, bin_adj_all, test_object, args))
        print("recursive accuracy: ", accuracy(r, labels).item())
        plot_if_possible(r)
        
        degrees = preds.sum(dim=1)
        preds = torch.diag(1./degrees)@preds
        mod_pred = make_modularity_matrix(preds)
        r = baseline_spectral(mod_pred, K)
        print('spectral', loss_fn(None, r, None, None, bin_adj_all, test_object, args))
        print("spectral accuracy: ", accuracy(r, labels).item())
        plot_if_possible(r)
    elif args.objective == 'kcenter':
        x = gonzalez_kcenter(dist_train, K)
        print('gonzalez train', obj_test(x))
        x = greedy_kcenter(dist_train, diameter, K)
        print('greedy train', obj_test(x))

7
agglomerative tensor(0.2021)
agglomerative accuracy:  0.14697193500738553
recursive tensor(0.0835)
recursive accuracy:  0.0672082717872969
spectral tensor(0.0357)
spectral accuracy:  0.06610044313146234


### Baseline optimization algorithms on full graph

In [53]:
if calculate_opt:
    if args.objective == 'modularity':
        preds = bin_adj_all
        r = greedy_modularity_communities(preds, K)
        print('agglomerative', loss_fn(None, r, None, None, bin_adj_all, test_object, args))
        print("agglomerative accuracy: ", accuracy(r, labels).item())
        plot_if_possible(r)
        
        r = partition(preds, K)
        print('recursive', loss_fn(None, r, None, None, bin_adj_all, test_object, args))
        print("recursive accuracy: ", accuracy(r, labels).item())
        plot_if_possible(r)

        degrees = preds.sum(dim=1)
        preds = torch.diag(1./degrees)@preds
        mod_pred = make_modularity_matrix(preds)
        r = baseline_spectral(mod_pred, K)
        print('spectral', loss_fn(None, r, None, None, bin_adj_all, test_object, args))
        print("spectral accuracy: ", accuracy(r, labels).item())
        plot_if_possible(r)
    elif args.objective == 'kcenter':
        x = gonzalez_kcenter(dist_all, K)
        print('gonzalez all', obj_test(x))
        x = greedy_kcenter(dist_all, diameter, K)
        print('greedy all', obj_test(x))

7
agglomerative tensor(0.0774)
agglomerative accuracy:  0.09601181683899557
recursive tensor(0.1977)
recursive accuracy:  0.1883308714918759
spectral tensor(0.1853)
spectral accuracy:  0.07533234859675036
